# Going beyond the GDP with the GrDP, factoring health and environmental costs in economic success

## Importing Libraries and Datasets

In [1]:
import pandas as pd
import numpy as np
from math import pi
import warnings
import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact
import plotly.graph_objects as go

In [2]:
External_costs = pd.read_csv('https://raw.githubusercontent.com/AntoineTrabia/Green-Domestic-Product/main/data/ExternalCostsDetailed.csv')
External_costs_total = pd.read_csv('https://raw.githubusercontent.com/AntoineTrabia/Green-Domestic-Product/main/data/ExternalCostsTotal.csv')
External_Costs_total_per_capita = pd.read_csv('https://raw.githubusercontent.com/AntoineTrabia/Green-Domestic-Product/main/data/ExternalCostsTotalPerCapita.csv')
GrDP = pd.read_csv('https://raw.githubusercontent.com/AntoineTrabia/Green-Domestic-Product/main/data/GrDP.csv')
GrDP_per_capita = pd.read_csv('https://raw.githubusercontent.com/AntoineTrabia/Green-Domestic-Product/main/data/GrDPPerCapita.csv')
Data_for_GrDP = pd.read_csv('https://raw.githubusercontent.com/AntoineTrabia/Green-Domestic-Product/main/data/DataForCalc.csv')
Data_for_calc = pd.read_csv('https://raw.githubusercontent.com/AntoineTrabia/Green-Domestic-Product/main/data/DataForEmissions.csv')
External_Costs_Percent_GDP = pd.read_csv('https://raw.githubusercontent.com/AntoineTrabia/Green-Domestic-Product/main/data/ExternalCostsPercentGDP.csv')

## **What is the GrDP ?**

The Gross Domestic Product (GDP) is a valuable indicator measuring the monetary value of all goods and
services produced in a country within a given time period. However, the GDP is not and never was an indicator of
economic performance and social progress. Already in 1934, Simon Kuznets, who invented the Gross National
Product (GNP) – GDP’s predecessor, warned that national income statistics do not measure welfare. Indeed,
while economic science is interested in properly managing all resources, the GDP does not encompass the
indirect impacts of productive activities such as environmental pollution. Thus, the GDP is not a good measure
of the value effectively created because it fails to account for the depletion of natural, social, and human capital
associated with economic activities.

Today, the climate crisis requires urgent action to decarbonise our society. However, there is often a perceived
dichotomy between promoting economic growth and protecting environmental sustainability. But these two
objectives need not be conflicting, as stated in 2009 by the Stiglitz-Sen-Fitoussi Commission:



”What we measure affects what we do; and if our measurements are flawed, decisions may be distorted.
Choices between promoting GDP and protecting the environment may be false choices, once environmental
degradation is appropriately included in our measurement of economic performance. So too, we often draw
inferences about what are good policies by looking at what policies have promoted economic growth; but if
our metrics of performance are flawed, so too may be the inferences that we draw.”
Stiglitz, Sen, Fitoussi et al., 2009. Report by the Commission on the Measurement of Economic Performance
and Social Progress. p7

In this paper, we propose a novel indicator, the Green Domestic Product (GrDP), to remedy some of the
shortcomings of GDP. The GrDP is calculated by subtracting the external costs associated with producing
goods and services from the standard measurement of GDP. This new measure was initially proposed by
Danthine et al. (2020) in the E4S white paper Green Domestic Product: Netting Greenhouse Gas Emissions
from Gross Domestic Product. We go one step further by extending the scope of GrDP to the emissions of air
pollutants and heavy metals in addition to the one of greenhouse gases (GHG). The impacts covered include
climate change, health issues, decrease in crops’ yields and biomass production, buildings degradation, and
damages to ecosystems due to eutrophication. A detailed description of the method used, data sources, and
assumptions is available in the report Green Domestic Product: Methodology1
.

We present in this paper an application of the GrDP framework to the case study of Switzerland to demonstrate
how GrDP can help understand the true costs and benefits of industrial activity. We first compare the evolution
of GDP, GrDP, and external costs. We then explore the decoupling between economic growth and environmental
pollution before proposing actions to leverage the co-benefits of decarbonation. We conclude by discussing the
limitations of the GrDP in its current version.

## **External costs are decreasing but remain significant**

### Explanation of different scenarios

In [3]:
# Function to plot data
def plot_country_data(country):
    plt.figure(figsize=(10, 6))
    plt.plot(Data_for_calc[Data_for_calc['countries'] == country]['Year'], Data_for_calc[Data_for_calc['countries'] == country]['Territorial GHG emissions [t]'], linestyle='-', label='Territorial Emissions')
    plt.plot(Data_for_calc[Data_for_calc['countries'] == country]['Year'], Data_for_calc[Data_for_calc['countries'] == country]['Residential GHG emissions [t]'], linestyle='-', label='Residential Emissions')
    plt.plot(Data_for_calc[Data_for_calc['countries'] == country]['Year'], Data_for_calc[Data_for_calc['countries'] == country]['Footprint GHG emissions [t]'], linestyle='-', label='Footprint Emissions')
    plt.title(f'GHG emissions of {country}')
    plt.xlabel('Year',size=14)
    plt.ylabel(f'tonnes of CO2eq',size=14)
    plt.grid(True)
    plt.legend()
    plt.ylim(bottom=0)
    plt.show()

# Get unique countries and columns
unique_countries = External_costs_total['countries'].unique()

# Create interactive plot using widgets
interact(plot_country_data, country=widgets.Dropdown(options=unique_countries, description='Country:'))

interactive(children=(Dropdown(description='Country:', options=('Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'G…

<function __main__.plot_country_data(country)>

In [4]:
def line_plot_emissions(fig):
  def create_trace(country, color):
      return go.Scatter(
          x=Data_for_calc[Data_for_calc['countries'] == country]['Year'],
          y=Data_for_calc[Data_for_calc['countries'] == country][f'{type} GHG emissions [t]'],
          name=f"{type.capitalize()} GHG Emissions",
          visible=(country == Data_for_calc.countries.unique()[0]),
          line=dict(color=color)
      )

  # Add Traces for each type of emissions
  for type, color in zip(['Territorial', 'Residential', 'Footprint'], ['#3182bd', '#31a354','#d95f0e']):
      for country in Data_for_calc.countries.unique():
          fig.add_trace(create_trace(country, color))

  # Create buttons for each country
  buttons = [dict(label=country,
                  method="update",
                  args=[{"visible": [country == c for c in Data_for_calc.countries.unique()]},
                        {"title": dict(text=f"{country} GHG Emissions", x=0.45, y=0.9),
                        "annotations": []}])
            for country in Data_for_calc.countries.unique()]

  # Update layout with buttons
  fig.update_layout(updatemenus=[dict(buttons=buttons, x=0, y=1.15)])

  # Set initial title
  fig.update_layout(
      updatemenus=[
          dict(
              active=0,
              buttons=buttons,
          )
      ],
      width=1200,
      height=700,  # Set height of the figure
      title=dict(text=f"{Data_for_calc.countries.unique()[0]} GHG Emissions", x=0.45, y=0.9),
      yaxis_title="Tonnes of CO2eq",
      xaxis_title='Year',
      yaxis=dict(rangemode='tozero'),
      xaxis=dict(range=[Data_for_calc['Year'].min() - 1, Data_for_calc['Year'].max()+1])
      )


  fig.show()

In [5]:
fig = go.Figure()
line_plot_emissions(fig)

### External costs vary between countries

In [6]:
# Function to plot data
def plot_bar_plot(year,column):

    data_year = External_Costs_total_per_capita.loc[External_Costs_total_per_capita['Year'] == year, ['countries', column]]

    countries = data_year['countries']
    values = data_year[column]

    sorted_data = sorted(zip(countries, values), key=lambda x: x[1], reverse=True)

    sorted_countries, sorted_costs_per_capita = zip(*sorted_data)

    plt.figure(figsize=(10, 6))
    plt.barh(sorted_countries, sorted_costs_per_capita, height=0.6)
    plt.xlabel('External costs per capita [Euro]')
    plt.ylabel('Countries')
    plt.title('External costs per capita by Country')
    plt.gca().invert_yaxis()  # Invert y-axis to have the highest GDP at the top
    plt.tight_layout()
    plt.show()

# Get unique years and columns
unique_years = External_Costs_total_per_capita['Year'].unique()
unique_columns = External_Costs_total_per_capita.columns[2:]  # Exclude 'countries' and 'Year'

# Create interactive plot using widgets
interact(plot_bar_plot, year=widgets.Dropdown(options=unique_years, description='Year:', value=2021), column=widgets.Dropdown(options=unique_columns, description='Scenario:', value='External costs per capita (Territorial & High GHG with VSL) scenario [Euro]'))

interactive(children=(Dropdown(description='Year:', index=31, options=(1990, 1991, 1992, 1993, 1994, 1995, 199…

<function __main__.plot_bar_plot(year, column)>

In [7]:
# Function to plot data
def plot_bar_plot(year,column):

    data_year = External_Costs_Percent_GDP.loc[External_Costs_Percent_GDP['Year'] == year, ['countries', column]]

    countries = data_year['countries']
    values = data_year[column]

    sorted_data = sorted(zip(countries, values), key=lambda x: x[1], reverse=True)

    sorted_countries, sorted_costs_per_capita = zip(*sorted_data)

    plt.figure(figsize=(10, 6))
    plt.barh(sorted_countries, sorted_costs_per_capita, height=0.6)
    plt.xlabel('External costs as % of GDP [%]')
    plt.ylabel('Countries')
    plt.title('External costs as % of GDP by Country')
    plt.gca().invert_yaxis()  # Invert y-axis to have the highest GDP at the top
    plt.tight_layout()
    plt.show()

# Get unique years and columns
unique_years = External_Costs_Percent_GDP['Year'].unique()
unique_columns = External_Costs_Percent_GDP.columns[2:]  # Exclude 'countries' and 'Year'

# Create interactive plot using widgets
interact(plot_bar_plot, year=widgets.Dropdown(options=unique_years, description='Year:', value=2021), column=widgets.Dropdown(options=unique_columns, description='Scenario:', value='(Territorial & High GHG with VSL) scenario [%]'))

interactive(children=(Dropdown(description='Year:', index=31, options=(1990, 1991, 1992, 1993, 1994, 1995, 199…

<function __main__.plot_bar_plot(year, column)>

### External costs are decreasing over time, but not all at the same rate

In [8]:
# Function to plot data
def plot_area_chart(country, column_ghg, column_airpol):

    AreaChart = (External_costs.loc[(External_costs["countries"]==country),
                            ["Year", column_ghg, column_airpol,'Heavy Metals (Total) [Euro]']])

    years = AreaChart['Year']
    ghg = AreaChart[column_ghg]
    air_pollutants = AreaChart[column_airpol]
    heavy_metals = AreaChart['Heavy Metals (Total) [Euro]']

    plt.figure(figsize=(12, 8))
    # Plotting
    plt.stackplot(years, ghg, air_pollutants, heavy_metals, labels=['GHG', 'Air Pollutants', 'Heavy Metals'], alpha=0.5)

    # Adding labels and title
    plt.xlabel('Year',size=16)
    plt.ylabel('External Costs [Euro]', size=16)
    plt.title(f"Evolution of external costs per group of pollutants in {country}, in Euros")
    plt.legend(loc='upper right')

    # Show plot
    plt.xticks(range(1990, 2022, 2), rotation=45)
    plt.tight_layout()
    plt.show()

# Get unique years and columns
unique_countries = External_costs['countries'].unique()  # Exclude 'countries' and 'Year'
column_list_ghg = ['GHG: Territorial & Low scenario [Euro]',
       'GHG: Territorial & Central scenario [Euro]',
       'GHG: Territorial & High scenario [Euro]',
       'GHG: Residential & Low scenario [Euro]',
       'GHG: Residential & Central scenario [Euro]',
       'GHG: Residential & High scenario [Euro]',
       'GHG: Footprint & Low scenario [Euro]',
       'GHG: Footprint & Central scenario [Euro]',
       'GHG: Footprint & High scenario [Euro]']
column_list_airpol = ['Air Pollutants - VOLY [Euro]', 'Air Pollutants - VSL [Euro]']

# Create interactive plot using widgets
interact(plot_area_chart, country=widgets.Dropdown(options=unique_countries, description='Country:'), column_ghg=widgets.Dropdown(options=column_list_ghg, description='GHG scenario:', value='GHG: Territorial & High scenario [Euro]'), column_airpol=widgets.Dropdown(options=column_list_airpol, description='Airpol scenario:',value='Air Pollutants - VSL [Euro]'))

interactive(children=(Dropdown(description='Country:', options=('Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'G…

<function __main__.plot_area_chart(country, column_ghg, column_airpol)>

In [9]:
# Function to plot data
def plot_area_chart(country, column_ghg, Health_scenario):

    num_colors = 6
    Oranges = plt.cm.Oranges(np.linspace(0.5, 1, num_colors))
    Greens = plt.cm.Greens(np.linspace(1, 0.5, num_colors))
    air_pollutant_colors = Oranges
    heavy_metal_colors = Greens

    if Health_scenario == "VOLY":
      AreaChart = (External_costs.loc[(External_costs["countries"]==country),
                            ["Year", column_ghg, 'Nox (Total - VOLY) [Euro]', 'PM2.5 (Health - VOLY) [Euro]', 'PM10 (Health - VOLY) [Euro]',
                 'SOx (Total - VOLY) [Euro]', 'NMVOC (Total - VOLY) [Euro]', 'NH3 (Total - VOLY) [Euro]','Pb (Total) [Euro]', 'Hg (Total) [Euro]', 'Cd (Total) [Euro]',
       'As (Total) [Euro]', 'Ni (Total) [Euro]', 'Cr (Total) [Euro]']])

      years = AreaChart['Year']
      ghg = AreaChart[column_ghg]

      Nox = AreaChart['Nox (Total - VOLY) [Euro]']
      PM2_5 = AreaChart['PM2.5 (Health - VOLY) [Euro]']
      PM10 = AreaChart['PM10 (Health - VOLY) [Euro]']
      SOx = AreaChart['SOx (Total - VOLY) [Euro]']
      NMVOC = AreaChart['NMVOC (Total - VOLY) [Euro]']
      NH3 = AreaChart['NH3 (Total - VOLY) [Euro]']

      Pb = AreaChart['Pb (Total) [Euro]']
      Hg = AreaChart['Hg (Total) [Euro]']
      Cd = AreaChart['Cd (Total) [Euro]']
      As = AreaChart['As (Total) [Euro]']
      Ni = AreaChart['Ni (Total) [Euro]']
      Cr = AreaChart['Cr (Total) [Euro]']

      plt.figure(figsize=(12, 8))

      plt.stackplot(years, ghg, Nox, PM2_5, PM10, SOx, NMVOC, NH3, Pb, Hg, Cd, As, Ni, Cr,
              labels=['GHG', 'NOx', 'PM2.5', 'PM10', 'SOx', 'NMVOC', 'NH3', 'Pb', 'Hg', 'Cd', 'As', 'Ni', 'Cr'],
              colors=['tab:blue', *air_pollutant_colors, *heavy_metal_colors],
              alpha=0.5)

      plt.xlabel('Year',size=16)
      plt.ylabel('External Costs [Euro]', size=16)
      plt.title(f"Evolution of external costs per pollutant in {country}, in Euros")
      plt.legend(loc='upper right')
      plt.xticks(range(1990, 2022, 2), rotation=45)
      plt.tight_layout()
      plt.show()

    if Health_scenario == "VSL":
      AreaChart = (External_costs.loc[(External_costs["countries"]==country),
                            ["Year", column_ghg, 'Nox (Total - VSL) [Euro]', 'PM2.5 (Health - VSL) [Euro]', 'PM10 (Health - VSL) [Euro]',
                 'SOx (Total - VSL) [Euro]', 'NMVOC (Total - VSL) [Euro]', 'NH3 (Total - VSL) [Euro]','Pb (Total) [Euro]', 'Hg (Total) [Euro]', 'Cd (Total) [Euro]',
       'As (Total) [Euro]', 'Ni (Total) [Euro]', 'Cr (Total) [Euro]']])

      years = AreaChart['Year']
      ghg = AreaChart[column_ghg]

      Nox = AreaChart['Nox (Total - VSL) [Euro]']
      PM2_5 = AreaChart['PM2.5 (Health - VSL) [Euro]']
      PM10 = AreaChart['PM10 (Health - VSL) [Euro]']
      SOx = AreaChart['SOx (Total - VSL) [Euro]']
      NMVOC = AreaChart['NMVOC (Total - VSL) [Euro]']
      NH3 = AreaChart['NH3 (Total - VSL) [Euro]']

      Pb = AreaChart['Pb (Total) [Euro]']
      Hg = AreaChart['Hg (Total) [Euro]']
      Cd = AreaChart['Cd (Total) [Euro]']
      As = AreaChart['As (Total) [Euro]']
      Ni = AreaChart['Ni (Total) [Euro]']
      Cr = AreaChart['Cr (Total) [Euro]']

      plt.figure(figsize=(12, 8))

      plt.stackplot(years, ghg, Nox, PM2_5, PM10, SOx, NMVOC, NH3, Pb, Hg, Cd, As, Ni, Cr,
              labels=['GHG', 'NOx', 'PM2.5', 'PM10', 'SOx', 'NMVOC', 'NH3', 'Pb', 'Hg', 'Cd', 'As', 'Ni', 'Cr'],
              colors=['tab:blue', *air_pollutant_colors, *heavy_metal_colors],
              alpha=0.5)

      plt.xlabel('Year',size=16)
      plt.ylabel('External Costs [Euro]', size=16)
      plt.title(f"Evolution of external costs per pollutant in {country}, in Euros")
      plt.legend(loc='upper right')
      plt.xticks(range(1990, 2022, 2), rotation=45)
      plt.tight_layout()
      plt.show()

# Get unique years and columns
unique_countries = External_costs['countries'].unique()  # Exclude 'countries' and 'Year'
column_list_ghg = ['GHG: Territorial & Low scenario [Euro]',
       'GHG: Territorial & Central scenario [Euro]',
       'GHG: Territorial & High scenario [Euro]',
       'GHG: Residential & Low scenario [Euro]',
       'GHG: Residential & Central scenario [Euro]',
       'GHG: Residential & High scenario [Euro]',
       'GHG: Footprint & Low scenario [Euro]',
       'GHG: Footprint & Central scenario [Euro]',
       'GHG: Footprint & High scenario [Euro]']
column_list_health = ['VOLY', 'VSL']
interact(plot_area_chart, country=widgets.Dropdown(options=unique_countries, description='Country:'), column_ghg=widgets.Dropdown(options=column_list_ghg, description='GHG scenario:',value='GHG: Territorial & High scenario [Euro]'), Health_scenario=widgets.Dropdown(options=column_list_health, description='Health scenario:',value='VSL'))

interactive(children=(Dropdown(description='Country:', options=('Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'G…

<function __main__.plot_area_chart(country, column_ghg, Health_scenario)>

## **The GrDP of Europeans countries**

### How does the GrDP compare to the GDP ?

In [10]:
# Function to plot data
def plot_country_data(country, column):
    plt.figure(figsize=(10, 6))
    plt.plot(Data_for_GrDP[Data_for_GrDP['countries'] == country]['Year'], Data_for_GrDP[Data_for_GrDP['countries'] == country]['GDP [EURO]'], linestyle='-', label='GDP')
    plt.plot(External_costs_total[External_costs_total['countries'] == country]['Year'], External_costs_total[External_costs_total['countries'] == country][column], linestyle='-', label='External Costs')
    plt.title(f'GDP and External Costs for {country}')
    plt.xlabel('Year',size=14)
    plt.ylabel(f'Euros',size=14)
    plt.grid(True)
    plt.legend()
    plt.ylim(bottom=0)
    plt.show()

# Get unique countries and columns
unique_countries = External_costs_total['countries'].unique()
unique_columns = External_costs_total.columns[2:]  # Exclude 'countries' and 'Year'

# Create interactive plot using widgets
interact(plot_country_data, country = External_costs_total['countries'].unique(), column=widgets.Dropdown(options=unique_columns, description='Scenario:', value='External costs (Territorial & High GHG with VSL) scenario [Euro]'))

interactive(children=(Dropdown(description='country', options=('Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'Ge…

<function __main__.plot_country_data(country, column)>

In [11]:
# Function to plot data
def plot_country_data(country, column):
    plt.figure(figsize=(10, 6))
    plt.plot(Data_for_GrDP[Data_for_GrDP['countries'] == country]['Year'], Data_for_GrDP[Data_for_GrDP['countries'] == country]['GDP [MIO_EURO]'], linestyle='-', label='GDP')
    plt.plot(GrDP[GrDP['countries'] == country]['Year'], GrDP[GrDP['countries'] == country][column], linestyle='-', label='GrDP')
    plt.title(f'GDP and GrDP for {country}')
    plt.xlabel('Year',size=14)
    plt.ylabel(f'Million Euros',size=14)
    plt.grid(True)
    plt.legend()
    min_GrDP_value = np.min(GrDP[GrDP['countries'] == country][column])
    plt.ylim(bottom=min(0, min_GrDP_value))
    plt.show()

# Get unique countries and columns
unique_countries = GrDP['countries'].unique()
unique_columns = GrDP.columns[2:]  # Exclude 'countries' and 'Year'

# Create interactive plot using widgets
interact(plot_country_data, country=widgets.Dropdown(options=unique_countries, description='Country:'), column=widgets.Dropdown(options=unique_columns, description='Scenario:', value='GrDP (Territorial & High GHG with VSL) scenario [Million Euro]'))

interactive(children=(Dropdown(description='Country:', options=('Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'G…

<function __main__.plot_country_data(country, column)>

### Comparison between countries

In [12]:
# Function to plot data
def plot_bar_plot(year,column):

    data_year = GrDP_per_capita.loc[GrDP_per_capita['Year'] == year, ['countries', column]]

    countries = data_year['countries']
    values = data_year[column]

    sorted_data = sorted(zip(countries, values), key=lambda x: x[1], reverse=True)

    sorted_countries, sorted_GDP_per_capita = zip(*sorted_data)

    plt.figure(figsize=(10, 6))
    plt.barh(sorted_countries, sorted_GDP_per_capita, height=0.6)
    plt.xlabel('GrDP per capita [Euro]')
    plt.ylabel('Countries')
    plt.title('GrDP per capita by Country')
    plt.gca().invert_yaxis()  # Invert y-axis to have the highest GDP at the top
    plt.tight_layout()
    plt.show()

# Get unique years and columns
unique_years = GrDP_per_capita['Year'].unique()
unique_columns = GrDP_per_capita.columns[2:]  # Exclude 'countries' and 'Year'

# Create interactive plot using widgets
interact(plot_bar_plot, year=widgets.Dropdown(options=unique_years, description='Year:', value=2021), column=widgets.Dropdown(options=unique_columns, description='Scenario:', value='GrDP per capita (Territorial & High GHG with VSL) scenario [Euro]'))

interactive(children=(Dropdown(description='Year:', index=26, options=(1995, 1996, 1997, 1998, 1999, 2000, 200…

<function __main__.plot_bar_plot(year, column)>

In [13]:
from ipywidgets import interact, Dropdown

def assign_value(scenario):
    global variable
    variable = scenario

variable = 'NINA'

# List of options for the dropdown menu
options = External_Costs_total_per_capita.columns[2:]

# Create the dropdown widget
dropdown = Dropdown(options=options)

# Define the interaction
interact(assign_value, scenario=dropdown)

# Update the variable whenever the dropdown value changes
dropdown.observe(lambda change: assign_value(change.new), names='value')


interactive(children=(Dropdown(description='scenario', options=('External costs per capita (Territorial & Low …

In [14]:
variable

'External costs per capita (Territorial & Low GHG with VOLY) scenario [Euro]'

In [15]:
def plot_map(year, column):
    fig = px.choropleth(External_Costs_total_per_capita,
                        locations='countries',
                        locationmode='country names',
                        color=column,
                        hover_name='countries',
                        animation_frame='Year',
                        title=f"{column} by Country",
                        color_continuous_scale=px.colors.sequential.Greens,
                        scope='europe',
                        height=800,
                       width=1000 )

    # Remove color bar title
    fig.update_coloraxes(colorbar_title=None)

    fig.show()


# Example usage
plot_map(2021, variable)

## **Can we decouple economic growth and health/environmental externalities**

## **Conclusion**